# Demo Minggu 2: Sampling, Kuantisasi, dan Analisis Waktu
## Sampling, Quantization, and Time-Domain Analysis

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

---

### Tujuan Pembelajaran

Setelah mengikuti demo ini, mahasiswa diharapkan dapat:

1. Memahami **Teorema Nyquist** dan syarat sampling yang benar
2. Mengenali fenomena **aliasing** ketika sampling rate terlalu rendah
3. Memahami efek **kuantisasi** (quantization) pada berbagai bit depth
4. Menghitung statistik sinyal di domain waktu (*time-domain analysis*)
5. Menerapkan **moving average filter** untuk pengurangan noise
6. Melakukan **analisis berjendela** (*windowed analysis*) pada sinyal biomedis

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_signals
from medsinyal.filters import moving_average

# Konfigurasi plot
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

print('Semua library berhasil dimuat.')

---
## 1. Teorema Nyquist (Nyquist Theorem)

**Teorema Nyquist-Shannon** menyatakan bahwa sinyal analog dapat direkonstruksi secara sempurna dari sampel-sampelnya jika **laju sampling** (*sampling rate*) minimal dua kali frekuensi tertinggi dalam sinyal:

$$f_s \geq 2 \cdot f_{\max}$$

Frekuensi $f_N = 2 \cdot f_{\max}$ disebut **Nyquist rate**, dan $f_{\max}$ disebut **frekuensi Nyquist**.

### Mengapa penting di biomedis?

| Sinyal | Frekuensi Tipikal | Sampling Rate Umum |
|---|---|---|
| ECG | 0.05 -- 150 Hz | 250 -- 1000 Hz |
| EEG | 0.5 -- 100 Hz | 256 -- 512 Hz |
| EMG | 10 -- 500 Hz | 1000 -- 2000 Hz |

Mari kita lihat efek sampling rate yang berbeda pada sinyal sinus 50 Hz.

In [ ]:
# Muat data sampling demo
data = load_synthetic('sampling_demo')

# Data berisi sinyal "analog" (resolusi tinggi, fs=10000 Hz)
# dan sinyal yang disampling pada berbagai sampling rate
print('Keys dalam data:', list(data.keys()))
print(f'Frekuensi sinyal: {float(data["f_signal"])} Hz')
print(f'Nyquist rate: {2 * float(data["f_signal"])} Hz')

In [ ]:
# Sinyal analog (referensi resolusi tinggi)
t_analog = data['t_analog']
analog_signal = data['analog_signal']
f_signal = float(data['f_signal'])

# Sampling rates: 500 Hz, 200 Hz, 120 Hz, 80 Hz
sampling_rates = [500, 200, 120, 80]

fig, axes = plt.subplots(len(sampling_rates), 1, figsize=(14, 12), sharex=True)
fig.suptitle(f'Sampling Sinyal Sinus {int(f_signal)} Hz pada Berbagai Sampling Rate', 
             fontsize=14, fontweight='bold', y=1.01)

# Batasi tampilan ke 100 ms pertama agar terlihat jelas
t_display = 0.1  # detik

for i, fs_s in enumerate(sampling_rates):
    t_s = data[f't_{fs_s}']
    sig_s = data[f'fs_{fs_s}']
    
    # Tentukan apakah aliasing terjadi
    nyquist_freq = fs_s / 2
    is_aliased = fs_s < 2 * f_signal
    status = 'ALIASING' if is_aliased else 'OK (memenuhi Nyquist)'
    color = 'tab:red' if is_aliased else 'tab:blue'
    
    # Plot sinyal analog sebagai referensi
    mask_analog = t_analog <= t_display
    axes[i].plot(t_analog[mask_analog], analog_signal[mask_analog], 
                 'gray', alpha=0.4, linewidth=1, label='Sinyal analog (referensi)')
    
    # Plot sinyal ter-sampling
    mask_sampled = t_s <= t_display
    markerline, stemlines, baseline = axes[i].stem(
        t_s[mask_sampled], sig_s[mask_sampled], basefmt='gray')
    plt.setp(stemlines, color=color, linewidth=0.8)
    plt.setp(markerline, color=color, markersize=4)
    
    axes[i].set_title(f'fs = {fs_s} Hz | Nyquist freq = {nyquist_freq} Hz | {status}', 
                      fontsize=11, color='red' if is_aliased else 'black')
    axes[i].set_ylabel('Amplitudo')
    axes[i].set_xlim(0, t_display)
    if i == 0:
        axes[i].legend(loc='upper right')

axes[-1].set_xlabel('Waktu (detik)')
plt.tight_layout()
plt.show()

print(f'\nSinyal asli: {f_signal} Hz')
print(f'Nyquist rate (minimum sampling rate): {2 * f_signal} Hz')
print(f'\nKesimpulan:')
for fs_s in sampling_rates:
    status = 'Tidak aliasing' if fs_s >= 2 * f_signal else f'ALIASING! (frekuensi alias = |{f_signal} - {fs_s}| = {abs(f_signal - fs_s)} Hz)'
    print(f'  fs = {fs_s} Hz --> {status}')

---
## 2. Aliasing

Ketika *sampling rate* kurang dari Nyquist rate ($f_s < 2f_{\max}$), frekuensi tinggi akan "terlipat" (*fold*) dan muncul sebagai frekuensi rendah palsu. Ini disebut **aliasing**.

Frekuensi alias dapat dihitung dengan:

$$f_{\text{alias}} = |f_{\text{signal}} - k \cdot f_s|$$

di mana $k$ adalah bilangan bulat terdekat yang meminimumkan selisih.

### Demonstrasi Visual

Mari kita bandingkan sinyal yang di-reconstruct dari sampling rate yang cukup vs. yang terlalu rendah.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

t_display = 0.1  # tampilkan 100 ms

# --- Kasus 1: Sampling cukup (fs=200 Hz untuk sinyal 50 Hz) ---
fs_good = 200
t_good = data[f't_{fs_good}']
sig_good = data[f'fs_{fs_good}']
mask_good = t_good <= t_display
mask_analog = t_analog <= t_display

axes[0].plot(t_analog[mask_analog], analog_signal[mask_analog], 
             'b-', alpha=0.3, label=f'Sinyal asli ({int(f_signal)} Hz)')
axes[0].plot(t_good[mask_good], sig_good[mask_good], 
             'bo-', markersize=5, label=f'Sampel (fs={fs_good} Hz)')
# Interpolasi linear untuk menunjukkan rekonstruksi
t_interp = np.linspace(0, t_display, 2000)
sig_interp = np.interp(t_interp, t_good[mask_good], sig_good[mask_good])
axes[0].plot(t_interp, sig_interp, 'g--', alpha=0.6, label='Rekonstruksi (interpolasi linear)')
axes[0].set_title(f'Sampling Cukup: fs={fs_good} Hz >= 2 x {int(f_signal)} Hz', fontsize=12)
axes[0].set_ylabel('Amplitudo')
axes[0].legend(loc='upper right')

# --- Kasus 2: Sampling kurang / aliasing (fs=80 Hz untuk sinyal 50 Hz) ---
fs_bad = 80
t_bad = data[f't_{fs_bad}']
sig_bad = data[f'fs_{fs_bad}']
mask_bad = t_bad <= t_display

# Frekuensi alias: sinyal 50 Hz muncul sebagai |50 - 80| = 30 Hz
f_alias = abs(f_signal - fs_bad)
alias_signal = np.sin(2 * np.pi * f_alias * t_analog[mask_analog])

axes[1].plot(t_analog[mask_analog], analog_signal[mask_analog], 
             'b-', alpha=0.3, label=f'Sinyal asli ({int(f_signal)} Hz)')
axes[1].plot(t_bad[mask_bad], sig_bad[mask_bad], 
             'ro-', markersize=5, label=f'Sampel (fs={fs_bad} Hz)')
axes[1].plot(t_analog[mask_analog], alias_signal, 
             'r--', alpha=0.7, linewidth=2, label=f'Sinyal alias ({int(f_alias)} Hz)')
axes[1].set_title(f'ALIASING: fs={fs_bad} Hz < 2 x {int(f_signal)} Hz --> alias pada {int(f_alias)} Hz', 
                   fontsize=12, color='red')
axes[1].set_ylabel('Amplitudo')
axes[1].set_xlabel('Waktu (detik)')
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

print('Perhatikan pada plot bawah: titik-titik sampel merah terlihat mengikuti')
print(f'gelombang merah putus-putus ({int(f_alias)} Hz), bukan gelombang biru asli ({int(f_signal)} Hz).')
print('Inilah yang disebut aliasing -- informasi frekuensi asli telah hilang!')

---
## 3. Efek Kuantisasi (Quantization Effects)

Setelah sampling (diskretisasi waktu), langkah berikutnya dalam ADC adalah **kuantisasi** -- diskretisasi amplitudo.

Jumlah level kuantisasi ditentukan oleh **bit depth**:

$$L = 2^b$$

| Bit Depth | Jumlah Level | Contoh Penggunaan |
|---|---|---|
| 8-bit | 256 | Audio kualitas rendah |
| 12-bit | 4096 | Alat ECG portabel |
| 16-bit | 65536 | Audio CD, alat medis presisi |
| 24-bit | 16777216 | Instrumen lab presisi tinggi |

**Quantization error** (kesalahan kuantisasi):

$$e[n] = x[n] - x_q[n]$$

di mana $x_q[n]$ adalah sinyal terkuantisasi.

In [ ]:
def quantize(signal, n_bits):
    """Kuantisasi sinyal ke sejumlah bit tertentu.
    
    Parameters
    ----------
    signal : np.ndarray
        Sinyal input.
    n_bits : int
        Jumlah bit kuantisasi.
    
    Returns
    -------
    np.ndarray
        Sinyal terkuantisasi.
    """
    n_levels = 2 ** n_bits
    # Normalisasi ke [0, 1]
    sig_min, sig_max = signal.min(), signal.max()
    sig_norm = (signal - sig_min) / (sig_max - sig_min)
    # Kuantisasi: bulatkan ke level terdekat
    sig_quant = np.round(sig_norm * (n_levels - 1)) / (n_levels - 1)
    # Kembalikan ke rentang asli
    return sig_quant * (sig_max - sig_min) + sig_min


# Buat sinyal sinus bersih
fs_q = 1000  # Hz
t_q = np.arange(0, 0.1, 1.0 / fs_q)  # 100 ms
sinyal_bersih = np.sin(2 * np.pi * 10 * t_q)  # sinus 10 Hz

# Kuantisasi pada berbagai bit depth
bit_depths = [8, 4, 2]

fig, axes = plt.subplots(len(bit_depths), 2, figsize=(14, 10))
fig.suptitle('Efek Kuantisasi pada Berbagai Bit Depth', fontsize=14, fontweight='bold')

for i, bits in enumerate(bit_depths):
    n_levels = 2 ** bits
    sig_q = quantize(sinyal_bersih, bits)
    error = sinyal_bersih - sig_q
    
    max_err = np.max(np.abs(error))
    rmse = np.sqrt(np.mean(error**2))
    
    # Plot sinyal asli vs terkuantisasi
    axes[i, 0].plot(t_q * 1000, sinyal_bersih, 'b-', alpha=0.5, linewidth=1, label='Asli')
    axes[i, 0].step(t_q * 1000, sig_q, 'r-', linewidth=1.2, where='mid', label=f'{bits}-bit')
    axes[i, 0].set_title(f'Kuantisasi {bits}-bit ({n_levels} level)')
    axes[i, 0].set_ylabel('Amplitudo')
    axes[i, 0].legend(loc='upper right', fontsize=9)
    
    # Plot quantization error
    axes[i, 1].plot(t_q * 1000, error, 'g-', linewidth=0.8)
    axes[i, 1].axhline(y=0, color='gray', linestyle='--', linewidth=0.5)
    axes[i, 1].set_title(f'Quantization Error ({bits}-bit) | Max={max_err:.4f}, RMSE={rmse:.4f}')
    axes[i, 1].set_ylabel('Error')

axes[-1, 0].set_xlabel('Waktu (ms)')
axes[-1, 1].set_xlabel('Waktu (ms)')
plt.tight_layout()
plt.show()

print('Ringkasan Quantization Error:')
print(f'{"Bit Depth":<12} {"Level":<10} {"Max Error":<15} {"RMSE":<15}')
print('-' * 52)
for bits in bit_depths:
    sig_q = quantize(sinyal_bersih, bits)
    error = sinyal_bersih - sig_q
    print(f'{bits:<12} {2**bits:<10} {np.max(np.abs(error)):<15.6f} {np.sqrt(np.mean(error**2)):<15.6f}')

### Pengamatan

- **8-bit** (256 level): Error sangat kecil, sinyal terkuantisasi hampir tidak bisa dibedakan dari sinyal asli.
- **4-bit** (16 level): Tangga kuantisasi mulai terlihat, namun bentuk sinyal masih jelas.
- **2-bit** (4 level): Sinyal sangat terdistorsi -- hanya ada 4 level amplitudo.

Dalam alat medis, **bit depth minimal 12-bit** biasanya digunakan untuk menjaga kualitas diagnostik sinyal.

---
## 4. Analisis Waktu (Time-Domain Analysis)

Analisis domain waktu (*time-domain analysis*) mengekstrak fitur statistik langsung dari sinyal tanpa transformasi ke domain frekuensi.

### Ukuran Statistik Penting

| Statistik | Rumus | Keterangan |
|---|---|---|
| **Mean** ($\mu$) | $\frac{1}{N}\sum x_i$ | Nilai rata-rata sinyal |
| **Variance** ($\sigma^2$) | $\frac{1}{N}\sum (x_i - \mu)^2$ | Sebaran/variabilitas sinyal |
| **RMS** | $\sqrt{\frac{1}{N}\sum x_i^2}$ | Root Mean Square, ukuran "kekuatan" sinyal |
| **Energy** ($E$) | $\sum x_i^2$ | Total energi sinyal |
| **Zero-Crossing Rate** (ZCR) | $\frac{1}{N-1}\sum |\text{sgn}(x_i) - \text{sgn}(x_{i-1})|/2$ | Laju perubahan tanda, indikator frekuensi |

In [ ]:
# Muat data ECG sintetik
ecg_data = load_synthetic('synthetic_ecg')

ecg_clean = ecg_data['ecg_clean']
ecg_noisy = ecg_data['ecg_noisy']
fs_ecg = float(ecg_data['fs'])
t_ecg = ecg_data['t']

print(f'Sampling rate: {fs_ecg} Hz')
print(f'Durasi: {t_ecg[-1]:.1f} detik')
print(f'Jumlah sampel: {len(ecg_clean)}')

# Visualisasi ECG bersih dan bernoise
fig = plot_signals(t_ecg, {
    'ECG Bersih (mV)': ecg_clean,
    'ECG Bernoise (mV)': ecg_noisy
}, title='Sinyal ECG Sintetik')
plt.show()

In [ ]:
def compute_signal_stats(signal):
    """Hitung statistik deskriptif dari sinyal.
    
    Parameters
    ----------
    signal : np.ndarray
        Input sinyal.
    
    Returns
    -------
    dict
        Dictionary berisi mean, variance, rms, energy, zcr.
    """
    N = len(signal)
    mean = np.mean(signal)
    variance = np.var(signal)
    rms = np.sqrt(np.mean(signal ** 2))
    energy = np.sum(signal ** 2)
    # Zero-crossing rate: jumlah perubahan tanda dibagi (N-1)
    zero_crossings = np.sum(np.abs(np.diff(np.sign(signal))) > 0)
    zcr = zero_crossings / (N - 1)
    
    return {
        'Mean': mean,
        'Variance': variance,
        'RMS': rms,
        'Energy': energy,
        'Zero-Crossing Rate': zcr
    }


# Hitung statistik untuk kedua sinyal
stats_clean = compute_signal_stats(ecg_clean)
stats_noisy = compute_signal_stats(ecg_noisy)

# Tampilkan sebagai tabel
print(f'{"Statistik":<25} {"ECG Bersih":>15} {"ECG Bernoise":>15}')
print('=' * 55)
for key in stats_clean:
    print(f'{key:<25} {stats_clean[key]:>15.6f} {stats_noisy[key]:>15.6f}')

print('\nPengamatan:')
print('- Variance ECG bernoise lebih besar karena adanya komponen noise.')
print('- Zero-crossing rate lebih tinggi pada sinyal bernoise (noise menambah fluktuasi).')
print('- Energy lebih tinggi pada sinyal bernoise karena noise menambah daya sinyal.')

---
## 5. Moving Average Filter untuk Reduksi Noise

**Moving average** adalah filter sederhana yang menghaluskan sinyal dengan menghitung rata-rata dari $M$ sampel berurutan:

$$y[n] = \frac{1}{M} \sum_{k=0}^{M-1} x[n-k]$$

### Trade-off:
- **Window kecil** ($M$ kecil): sedikit smoothing, fitur tajam terjaga
- **Window besar** ($M$ besar): banyak smoothing, tapi fitur tajam (seperti QRS complex) bisa terdistorsi

Kita akan menerapkan `moving_average()` dari library `medsinyal` pada sinyal ECG bernoise.

In [ ]:
# Uji beberapa window size
window_sizes = [5, 15, 50]

fig, axes = plt.subplots(len(window_sizes) + 1, 1, figsize=(14, 14), sharex=True)
fig.suptitle('Moving Average Filter pada ECG Bernoise', fontsize=14, fontweight='bold')

# Tampilkan 3 detik pertama agar detail terlihat
t_show = 3.0  # detik
mask_show = t_ecg <= t_show

# Plot sinyal bernoise (tanpa filter)
axes[0].plot(t_ecg[mask_show], ecg_noisy[mask_show], 'r-', alpha=0.7, linewidth=0.8)
axes[0].set_title('ECG Bernoise (Tanpa Filter)', fontsize=11)
axes[0].set_ylabel('Amplitudo (mV)')

for i, M in enumerate(window_sizes):
    ecg_filtered = moving_average(ecg_noisy, M)
    
    axes[i + 1].plot(t_ecg[mask_show], ecg_clean[mask_show], 
                     'b-', alpha=0.3, linewidth=0.8, label='ECG Bersih (referensi)')
    axes[i + 1].plot(t_ecg[mask_show], ecg_filtered[mask_show], 
                     'g-', linewidth=1.2, label=f'Moving Average (M={M})')
    axes[i + 1].set_title(f'Moving Average, Window Size M = {M} ({M/fs_ecg*1000:.1f} ms)', fontsize=11)
    axes[i + 1].set_ylabel('Amplitudo (mV)')
    axes[i + 1].legend(loc='upper right', fontsize=9)

axes[-1].set_xlabel('Waktu (detik)')
plt.tight_layout()
plt.show()

# Hitung RMSE terhadap sinyal bersih
print('\nPerbandingan RMSE terhadap sinyal bersih:')
rmse_noisy = np.sqrt(np.mean((ecg_noisy - ecg_clean) ** 2))
print(f'  Tanpa filter:  RMSE = {rmse_noisy:.6f}')
for M in window_sizes:
    ecg_filtered = moving_average(ecg_noisy, M)
    rmse_filtered = np.sqrt(np.mean((ecg_filtered - ecg_clean) ** 2))
    improvement = (1 - rmse_filtered / rmse_noisy) * 100
    print(f'  M = {M:>3}:        RMSE = {rmse_filtered:.6f}  (perbaikan {improvement:.1f}%)')

### Pengamatan Moving Average

- **M = 5**: Sedikit pengurangan noise, bentuk gelombang ECG terjaga dengan baik.
- **M = 15**: Pengurangan noise cukup baik, puncak R sedikit melebar.
- **M = 50**: Noise sangat berkurang, tapi QRS complex sudah terdistorsi signifikan (amplitudo menurun, lebar bertambah).

Untuk sinyal ECG, moving average **bukan** filter terbaik karena tidak selektif terhadap frekuensi. Filter yang lebih baik untuk ECG antara lain:
- **Bandpass filter** (misalnya 0.5--40 Hz untuk ECG diagnostik)
- **Notch filter** untuk menghilangkan interferensi jala-jala listrik (50/60 Hz)

---
## 6. Analisis Berjendela (Windowed Analysis)

Sinyal biomedis sering bersifat **non-stationary** -- sifat statistiknya berubah seiring waktu. Untuk menangkap perubahan ini, kita menghitung statistik dalam **jendela geser** (*sliding window*).

Prinsipnya:
1. Bagi sinyal menjadi segmen-segmen yang overlap
2. Hitung statistik pada setiap segmen
3. Plot hasilnya sebagai fungsi waktu

Ini berguna misalnya untuk:
- Mendeteksi perubahan heart rate pada ECG
- Mengidentifikasi artefak pada EEG
- Monitoring energi sinyal EMG selama kontraksi otot

In [ ]:
def windowed_analysis(signal, fs, window_sec=1.0, overlap=0.5):
    """Hitung statistik dalam sliding window.
    
    Parameters
    ----------
    signal : np.ndarray
        Input sinyal.
    fs : float
        Sampling rate (Hz).
    window_sec : float
        Ukuran window dalam detik.
    overlap : float
        Fraksi overlap antar window (0.0 -- 1.0).
    
    Returns
    -------
    dict
        Dictionary berisi t_windows, rms, energy, zcr, variance.
    """
    window_samples = int(window_sec * fs)
    step = int(window_samples * (1 - overlap))
    
    t_windows = []
    rms_vals = []
    energy_vals = []
    zcr_vals = []
    var_vals = []
    
    for start in range(0, len(signal) - window_samples + 1, step):
        segment = signal[start:start + window_samples]
        t_center = (start + window_samples / 2) / fs
        
        t_windows.append(t_center)
        rms_vals.append(np.sqrt(np.mean(segment ** 2)))
        energy_vals.append(np.sum(segment ** 2))
        zc = np.sum(np.abs(np.diff(np.sign(segment))) > 0)
        zcr_vals.append(zc / (len(segment) - 1))
        var_vals.append(np.var(segment))
    
    return {
        't_windows': np.array(t_windows),
        'rms': np.array(rms_vals),
        'energy': np.array(energy_vals),
        'zcr': np.array(zcr_vals),
        'variance': np.array(var_vals)
    }


# Analisis berjendela pada ECG bersih
results = windowed_analysis(ecg_clean, fs_ecg, window_sec=0.5, overlap=0.5)

fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)
fig.suptitle('Analisis Berjendela pada ECG Bersih (window=0.5s, overlap=50%)', 
             fontsize=14, fontweight='bold')

# Plot sinyal asli
axes[0].plot(t_ecg, ecg_clean, 'b-', linewidth=0.8)
axes[0].set_ylabel('ECG (mV)')
axes[0].set_title('Sinyal ECG Bersih')

# RMS
axes[1].plot(results['t_windows'], results['rms'], 'g-o', markersize=3)
axes[1].set_ylabel('RMS')
axes[1].set_title('Root Mean Square (RMS) per Window')

# Energy
axes[2].plot(results['t_windows'], results['energy'], 'm-o', markersize=3)
axes[2].set_ylabel('Energy')
axes[2].set_title('Energy per Window')

# Zero-Crossing Rate
axes[3].plot(results['t_windows'], results['zcr'], 'r-o', markersize=3)
axes[3].set_ylabel('ZCR')
axes[3].set_title('Zero-Crossing Rate per Window')
axes[3].set_xlabel('Waktu (detik)')

plt.tight_layout()
plt.show()

print('Pengamatan:')
print('- RMS dan Energy menunjukkan pola periodik yang sesuai dengan detak jantung.')
print('- Window yang mengandung QRS complex memiliki RMS dan Energy tinggi.')
print('- Zero-crossing rate juga bervariasi -- meningkat saat QRS (frekuensi tinggi).')

In [ ]:
# Perbandingan windowed analysis: ECG bersih vs bernoise
results_clean = windowed_analysis(ecg_clean, fs_ecg, window_sec=0.5, overlap=0.5)
results_noisy = windowed_analysis(ecg_noisy, fs_ecg, window_sec=0.5, overlap=0.5)

fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
fig.suptitle('Perbandingan Windowed Analysis: ECG Bersih vs Bernoise', 
             fontsize=14, fontweight='bold')

metrics = ['rms', 'energy', 'variance']
labels = ['RMS', 'Energy', 'Variance']
colors_clean = ['blue', 'blue', 'blue']
colors_noisy = ['red', 'red', 'red']

for i, (metric, label) in enumerate(zip(metrics, labels)):
    axes[i].plot(results_clean['t_windows'], results_clean[metric], 
                 'b-o', markersize=3, label='ECG Bersih')
    axes[i].plot(results_noisy['t_windows'], results_noisy[metric], 
                 'r-o', markersize=3, alpha=0.7, label='ECG Bernoise')
    axes[i].set_ylabel(label)
    axes[i].set_title(f'{label} per Window')
    axes[i].legend(loc='upper right')

axes[-1].set_xlabel('Waktu (detik)')
plt.tight_layout()
plt.show()

print('Pengamatan:')
print('- Noise meningkatkan baseline dari semua metrik (RMS, Energy, Variance).')
print('- Pola periodik dari detak jantung masih terlihat pada sinyal bernoise,') 
print('  tetapi kurang jelas karena noise menambah variabilitas di setiap window.')
print('- Windowed analysis berguna untuk monitoring sinyal secara real-time.')

---
## 7. Kesimpulan

### Ringkasan Materi Minggu 2

1. **Teorema Nyquist**: Sampling rate harus minimal $2 \times f_{\max}$ untuk menghindari kehilangan informasi. Dalam praktik biomedis, sampling rate biasanya dipilih 3--10 kali frekuensi tertinggi sinyal.

2. **Aliasing**: Jika sampling rate kurang dari Nyquist rate, frekuensi tinggi muncul sebagai frekuensi rendah palsu. Aliasing **tidak dapat diperbaiki** setelah terjadi -- pencegahan dilakukan dengan **anti-aliasing filter** sebelum sampling.

3. **Kuantisasi**: Bit depth menentukan resolusi amplitudo. Lebih banyak bit = lebih kecil quantization error. Alat medis umumnya menggunakan minimal 12-bit ADC.

4. **Analisis Domain Waktu**: Statistik sederhana (mean, variance, RMS, energy, ZCR) memberikan informasi penting tentang karakteristik sinyal.

5. **Moving Average**: Filter sederhana untuk smoothing, tapi memiliki trade-off antara pengurangan noise dan distorsi fitur sinyal.

6. **Analisis Berjendela**: Memungkinkan pengamatan perubahan sifat statistik sinyal sepanjang waktu -- penting untuk sinyal non-stationary seperti ECG dan EEG.

### Referensi

- Rangayyan, R.M. *Biomedical Signal Analysis*, 2nd Ed. Wiley-IEEE Press, 2015.
- Sörnmo, L. & Laguna, P. *Bioelectrical Signal Processing in Cardiac and Neurological Applications*. Academic Press, 2005.
- Oppenheim, A.V. & Willsky, A.S. *Signals and Systems*, 2nd Ed. Prentice Hall, 1997.